In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from difflib import get_close_matches
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import seaborn as sns
from typing import Union, Iterable, Optional, Tuple, List

In [2]:
# Среднедушевые денежные доходы (в месяц), руб.
pcci = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', sheet_name=0)

# Реальные денежные доходы, в процентах к предыдущему году
ri = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', sheet_name=1)

# Реальная начисленная заработная плата, в процентах к предыдущему году
fwp = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', sheet_name=2)

# Среднемесячная номинальная начисленная заработная плата, руб.
rpay = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', sheet_name=3)

# Число умерших на первом году жизни детей за год, по всем регионам,, в сельской местности
cmr = pd.read_excel('data/child_mortality_rural_1990_2021.xls', sheet_name=0, skiprows=2)
cmr = cmr.rename(columns={'Unnamed: 0':'region'})
cmr = cmr.drop(cmr.columns[[1,2]], axis=1)

# Число умерших на первом году жизни детей за год, по всем регионам, в городской местности
cmu = pd.read_excel('data/child_mortality_urban_1990_2021.xls', sheet_name=0, skiprows=2)
cmu = cmu.rename(columns={'Unnamed: 0':'region'})
cmu = cmu.drop(cmu.columns[[1,2]], axis=1)

# Число людей с инвалидностью по регионам, по месяцам, по возрастным группам
dta = pd.read_csv('data/disabled_total_by_age_2017_2022.csv', sep = ',')

# Заболеваемость с впервые в жизни установленным диагнозом алкоголизма и алкогольного психоза на 100 тыс.  населения
alco_05_16 = pd.read_excel('data/drug_alco.xlsx', sheet_name=0)

alco_17_18 = pd.read_excel('data/drug_alco.xlsx', sheet_name=1)

# Заболеваемость с впервые в жизни установленным диагнозом наркомании на 100 тыс.  населения (человек, на 1 января)
drugs_05_16 = pd.read_excel('data/drug_alco.xlsx', sheet_name=2)

drugs_17_18 = pd.read_excel('data/drug_alco.xlsx', sheet_name=3)

# валовой региональный продукт на душу населения, в рублях.
grp = pd.read_excel('data/gross_regional_product_1996_2020.xls', sheet_name=0, skiprows=2)
grp = grp.rename(columns={'Unnamed: 0':'region'})

# Характеристика жилищных условий домохозяйств, % от всех домохозяйств
hc = pd.read_excel('data/housing_2020.xlsx', sheet_name=0)
hc = hc.rename(columns={'Регион':'region'})

# Оценка домохозяйствами состояния занимаемого ими жилого помещения, % от всех домохозяйств
hi = pd.read_excel('data/housing_2020.xlsx', sheet_name=1)
hi = hi.rename(columns={'Регион':'region'})

# Заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний
mad = pd.read_excel('data/morbidity_2005_2020_age_disease.xls', sheet_name=0, skiprows=2)
mad = mad.rename(columns={'Unnamed: 0': 'region'})
mad = mad.rename(columns={'Unnamed: 1': 'type'})
mad = mad.rename(columns={'Unnamed: 2': 'age'})

# Ежемесячное количество новорожденных
nm = pd.read_csv('data/newborn_2006_2022_monthly.csv', sep = ';')
nm = nm.rename(columns={'Region': 'region'})

# Численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг
pop = pd.read_excel('data/population.xlsx', sheet_name=0, skiprows=1)
pop = pop.rename(columns={'Unnamed: 0': 'region'})
pop = pop.drop(pop.columns[1], axis=1)

# Процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам
ppr = pd.read_csv('data/poverty_percent_by_regions_1992_2020.csv', sep = ',')

# Распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам
ps_2017 = pd.read_excel('data/poverty_socdem_2017.xls', sheet_name=0, skiprows=2)
ps_2017 = ps_2017.rename(columns={'Unnamed: 0': 'region'})

ps_2018 = pd.read_excel('data/poverty_socdem_2018.xls', sheet_name=0, skiprows=2)
ps_2018 = ps_2018.rename(columns={'Unnamed: 0': 'region'})

ps_2019 = pd.read_excel('data/poverty_socdem_2019.xls', sheet_name=0, skiprows=2)
ps_2019 = ps_2019.rename(columns={'Unnamed: 0': 'region'})

ps_2020 = pd.read_excel('data/poverty_socdem_2020.xls', sheet_name=0, skiprows=2)
ps_2020 = ps_2020.rename(columns={'Unnamed: 0': 'region'})

# Объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами
rp_05_16 = pd.read_csv('data/regional_production_2005_2016.csv', sep = ',')

rp_17_20 = pd.read_csv('data/regional_production_2017_2020.csv', sep = ',')

# Оборот розничной торговли на душу населения, в рублях.
rtpc = pd.read_excel('data/retail_turnover_per_capita_2000_2021.xls', sheet_name=0, skiprows=2)
rtpc = rtpc.rename(columns={'Unnamed: 0': 'region'})
rtpc = rtpc.drop(rtpc.columns[1], axis=1)

# Расходы на социальную политику от общих расходов бюджета региона, % в год
wes = pd.read_excel('data/welfare_expense_share_2015_2020.xlsx')

# 1. Преобразование и очистка данных

## 1.1. Преобразование данных

На данный момемнт имеем наборы исследований из 25 датасетов по всем регионам Российской Федерации.
Основная часть описывает статистикку тех или иных критериев населения в кажом регионе по годам (месяцам)

> В некоторых датасетах имеются дополнительные столбцы такие как *(тип заболевания, возраст, тип промышнленности и тп.)*

Приведём датасеты к единому типу ----> регион-год (или год_наименование признака)

### 1.1.0. В датасете dta (число людей с инвалюдностью по месяцам по возрастным группам)

> приведём датасет к виду year_age_group, для упрощения возьмем статистику по годам, данные на 01.01 i-го года

In [3]:
# извлекаем год из даты
dta['year'] = pd.to_datetime(dta['date'], errors='coerce').dt.year

# считаем укрупнённые группы
dta['до_60'] = dta[['18_30','31_40','41_50','51_60']].sum(axis=1)
dta['после_60'] = dta['60_']

# группировка и pivot: wide-формат
dta_out = (dta.groupby(['region','year'], as_index=False)[['до_60','после_60']].sum()
         .pivot(index='region', columns='year', values=['до_60','после_60']))

# переименуем колонки в формат 2017_до_60
dta_out.columns = [f"{y}_{g}" for g,y in dta_out.columns]
dta_out = dta_out.reset_index()

### 1.1.1. Соединение датасетов alco, drugs 

In [4]:
alco = alco_05_16.merge(alco_17_18, on = 'region', how = 'left')

In [5]:
drugs = drugs_05_16.merge(drugs_17_18, on = 'region', how = 'left')

### 1.1.2. В датасете mad помимо статистических данных присутствует 2 дополнительных признака (колонны)


In [6]:
mad.head()

,region,type,age,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Российская Федерация,"Беременность, роды и послеродовой период",0-14 лет,21.3,17.6,17.9,22.1,18.1,18.9,16.2,17.7,12.2,12.1,8.1,15.5
1,Российская Федерация,"Беременность, роды и послеродовой период",15-17 лет,1537.3,1527.5,1605.2,1681.5,1618.9,1451.3,1247.6,1229.1,1272.3,1186.5,914.0,794.0
2,Российская Федерация,"Беременность, роды и послеродовой период",18 лет и старше,6731.7,6833.5,7229.7,7516.9,8007.5,8084.4,7981.6,8141.8,8084.6,8219.9,7734.1,7278.6
3,Российская Федерация,"Беременность, роды и послеродовой период",Всего,5719.4,5861.2,6258.7,6556.6,7011.6,7085.3,6979.7,7121.5,7067.6,7145.3,6700.2,6291.1
4,Российская Федерация,Болезни глаза и его придаточного аппарата,0-14 лет,5643.4,5581.1,5760.8,5775.9,5743.6,5884.8,5931.9,6206.9,6104.5,5919.4,5910.3,5971.5


> объединим их со статистическими признаками по приципу type × age × year

In [7]:
# mad - заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний

# Преобразование типа заболеваемости и возрастной группы в столбцы (по годам)

stopwords = {"и", "в", "во", "на", "по", "при", "из", "к", "о", "об", "от", "до", "за", "у", "со", "с"}

# Функция преобразования названия заболевания в абривеатуру
def make_abbr(text: str) -> str:
    words = re.split(r"\s+", str(text).strip())
    abbr = "".join(w[0].upper() for w in words if w.lower() not in stopwords and w)
    return abbr

mad["abbr"] = mad["type"].apply(make_abbr)

# создаём словарь преобразования
age_map = {
    "0-14 лет": 1,
    "15-17 лет": 2,
    "18 лет и старше": 3,
    "Всего": 4
}

# применяем замену
mad["age_group"] = mad["age"].map(age_map)

mad["abbr"] = mad["type"].apply(make_abbr)

mad = mad.drop(['type', 'age'], axis = 1)


In [8]:
# melt: превращаем года в строки
mad_long = mad.melt(
    id_vars=["region", "abbr", "age_group"],
    value_vars=[str(y) for y in range(2005, 2017)],
    var_name="year",
    value_name="value"
)

# делаем имя столбца
mad_long["col_name"] = mad_long["abbr"] + "_" + mad_long["age_group"].astype(str) + "_" + mad_long["year"].astype(str)

# pivot: значения -> столбцы
mad_wide = mad_long.pivot_table(
    index="region",
    columns="col_name",
    values="value",
    aggfunc="first"
).reset_index()

# убираем имя в колонках (чтобы не было MultiIndex)
mad_wide.columns.name = None
mad_wide.head()

,region,БГЕПА_1_2005,БГЕПА_1_2006,БГЕПА_1_2007,БГЕПА_1_2008,БГЕПА_1_2009,БГЕПА_1_2010,БГЕПА_1_2011,БГЕПА_1_2012,БГЕПА_1_2013,...,ТОДПВВП_4_2007,ТОДПВВП_4_2008,ТОДПВВП_4_2009,ТОДПВВП_4_2010,ТОДПВВП_4_2011,ТОДПВВП_4_2012,ТОДПВВП_4_2013,ТОДПВВП_4_2014,ТОДПВВП_4_2015,ТОДПВВП_4_2016
0,Ненецкий автономный округ (Арханге...,12275.4,10127.5,10592.2,10238.7,16919.0,20198.2,15968.4,14759.3,12665.0,...,14155.9,14095.3,14649.7,13113.3,13533.2,13492.9,12571.0,13787.4,12518.4,12175.2
1,Ханты-Мансийский автономный округ ...,7131.0,6693.5,6648.7,6722.8,6974.8,6984.8,6244.6,6549.5,6486.8,...,10933.5,10862.1,10780.8,10689.4,11292.2,10989.4,10926.5,10517.2,9527.6,9416.9
2,Ямало-Ненецкий автономный округ (Т...,10678.1,8690.2,12121.1,11502.3,10395.6,11535.3,10433.4,9057.3,9797.1,...,10416.3,10098.8,9710.9,9679.9,10155.9,9814.8,10015.1,9574.3,8992.2,9012.1
3,Алтайский край,5771.6,5731.5,5693.8,5837.2,5676.6,5498.9,5560.7,5852.6,6022.8,...,7891.0,8030.3,7837.7,7912.7,8685.3,9656.0,10105.9,9447.1,10252.9,9288.2
4,Амурская область,5175.0,4976.5,4788.7,5093.9,4433.1,5018.7,5635.9,6435.5,5395.4,...,8227.0,8169.4,8056.0,8276.9,9305.5,9133.9,9161.7,9206.4,8499.3,8364.1


> После преобразований имеем столбцы заболевание_возрастная категория_год

***Выделим группу всех заболеваний всех людей ['ВЗ_4'] за каждый год***

In [9]:
cols = mad_wide.filter(like='ВЗ_4').columns
mad_wide = mad_wide[['region'] + list(cols)]

### 1.1.3. Переименование столбцов в  признаках nm и pop

> Пререведём месяца в наименовании колонн в числа для упрощения чтения

In [10]:
# nm - ежемесячное количество новорожденных
# pop - численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг

# словарь месяцев
month_map = {
    "январь": "01", "февраль": "02", "март": "03", "апрель": "04",
    "май": "05", "июнь": "06", "июль": "07", "август": "08",
    "сентябрь": "09", "октябрь": "10", "ноябрь": "11", "декабрь": "12"
}

def rename_cols(cols):
    new_cols = []
    for c in cols:
        if isinstance(c, str) and any(m in c.lower() for m in month_map):
            parts = c.replace("г.", "").strip().split()
            month = month_map.get(parts[0].lower(), None)
            year = [p for p in parts if p.isdigit()][0]
            if month:
                new_cols.append(f"{month}_{year}")
            else:
                new_cols.append(c)
        else:
            new_cols.append(c)
    return new_cols

nm.columns = rename_cols(nm.columns)

pop.columns = rename_cols(pop.columns)
cols = list(pop.columns)[1:]
pop= pop.rename(columns={c: c.replace('01_', '') for c in cols})

In [11]:
def copy_section_values_down(
    df: pd.DataFrame,
    label_col: str = "region",
    protect_cols=None,                # столбцы, которые НЕ трогаем
    section_regex: str = r"^\s*Раздел\b",
    drop_section_rows: bool = True    # удалить строки "Раздел ..."
) -> pd.DataFrame:
    out = df.copy()

    if protect_cols is None:
        protect_cols = [label_col]
    else:
        protect_cols = list(set([label_col] + list(protect_cols)))

    # маска строк "Раздел ..."
    s = out[label_col].astype(str)
    mask = s.str.match(section_regex)

    # столбцы для копирования: все числовые, кроме защищённых
    num_cols = [c for c in out.columns
                if c not in protect_cols and pd.api.types.is_numeric_dtype(out[c])]

    # если числовых не нашли (например, импорт как object) — копируем всё, кроме защищённых
    if not num_cols:
        num_cols = [c for c in out.columns if c not in protect_cols]

    # позиционно: копируем значения вниз (из "Раздел" → в следующую строку)
    section_idxs = out.index[mask]
    for idx in section_idxs:
        pos = out.index.get_loc(idx)
        if pos + 1 < len(out):
            nxt = out.index[pos + 1]
            out.loc[nxt, num_cols] = out.loc[idx, num_cols].values

    # по желанию удаляем строки "Раздел ..."
    if drop_section_rows:
        out = out[~mask].reset_index(drop=True)

    return out

In [12]:
pop = copy_section_values_down(pop, label_col="region", protect_cols=["code"])

In [13]:
num_cols = nm.select_dtypes(include="object").columns.drop("region")

for col in num_cols:
    nm[col] = (
        nm[col]
        .astype(str)
        .str.replace(r"\s+", "", regex=True)   # убираем пробелы
        .str.replace(",", ".", regex=False)    # запятая → точка
    )
    # переводим в число, "мусор" станет NaN
    nm[col] = pd.to_numeric(nm[col], errors="coerce")

In [14]:
def sum_by_year(df: pd.DataFrame, id_col: str = "region") -> pd.DataFrame:
    """
    Складывает значения по месяцам -> суммы за год.
    """
    # отделяем колонку регионов
    if id_col in df.columns:
        regions = df[id_col]
        vals = df.drop(columns=[id_col])
    else:
        regions = None
        vals = df

    # достаем год из названия колонок
    years = vals.columns.to_series().str.extract(r'((?:19|20)\d{2})', expand=False)

    # группируем и суммируем по годам
    year_sum = vals.groupby(years, axis=1).sum(min_count=1)

    # возвращаем регион + суммы по годам
    if regions is not None:
        year_sum.insert(0, id_col, regions)

    return year_sum

In [15]:
nm = sum_by_year(nm, id_col="region")
nm.sample(2)

C:\Users\User\AppData\Local\Temp\ipykernel_31520\2163069589.py:17: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  year_sum = vals.groupby(years, axis=1).sum(min_count=1)


,region,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
11,Московская область,60483.99,63452.99,70384.98,74137.99,76951.99,76487.0,83382.0,85385.99,90040.98,94687.99,96807.98,88882.99,82947.97,73730.99,79106.98,73520.99,28668.00
73,Ямало-Ненецкий автономный округ (Тюменская обл...,6924.00,7691.00,7986.00,8143.00,8309.00,8255.0,9025.0,8941.00,9164.00,8965.00,8261.00,7555.00,7219.00,6835.00,7092.00,6950.00,2813.99


### 1.1.4. Преобразование признаков - Характеристика жилищных условий домохозяйств

In [16]:
hi = hi.rename(columns={'из них домохозяйства, собирающиеся улучшить свои жилищные условия': 'hi_proc_НУЖУ', 'домохозяйства, не собирающиеся улучшать свои жилищные условия': 'hi_proc_УЖУ'})
hi = hi[['region', 'hi_proc_НУЖУ', 'hi_proc_УЖУ']]

In [17]:
hc = hc[['region',
         'в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности',
         'в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность',
         'в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность',
         'Размер жилой площади в расчете на члена домохозяйства']]
hc = hc.rename(columns={
    'в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности' : 'hc_proc_НИС',
    'в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность' : 'hc_proc_ИОС',
    'в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность' : 'hc_proc_ИБС',
    'Размер жилой площади в расчете на члена домохозяйства' : 'hc_ЖП'
})

hc['hc_proc_ИС'] = (hc['hc_proc_ИОС'] + hc['hc_proc_ИБС'])
hc = hc.drop(['hc_proc_ИБС', 'hc_proc_ИОС'], axis = 1)


### 1.1.5. Датасет ppr - 'Процент людей, живущих за чертой бедности'

> Переведём признак [year] в колонны по принципу year_poverty_percent

In [18]:
ppr_wide = ppr.pivot(
    index="region",
    columns="year",
    values="poverty_percent"
)

# переименуем колонки в нужный формат: year_poverty_percent
ppr_wide.columns = [f"{year}" for year in ppr_wide.columns]

ppr_wide = ppr_wide.reset_index()
ppr_wide.head()

,region,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Архангельская область (кроме Ненец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3
1,Ненецкий автономный округ (Арханге...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,37.9,...,7.7,6.5,7.8,8.9,9.4,10.1,10.6,9.7,9.4,9.4
2,Тюменская область (кроме Ханты-Ман...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.4,11.8,12.0,14.7,15.5,15.4,14.3,14.4,13.6
3,Ханты-Мансийский автономный округ ...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,11.8,...,10.6,9.8,10.4,11.1,11.9,11.6,9.5,9.0,8.9,8.4
4,Ямало-Ненецкий автономный округ (Т...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,11.1,...,7.4,6.4,6.4,6.7,7.5,7.1,6.1,5.8,5.5,4.8


### 1.1.6. Датасет *ps_[year]* - переименуем колонны, соединим таблицы по годам

In [19]:
# ps_[year] - распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам

def rename_cols_ps(df):
    # удаляем колонку
    df = df.drop(columns=["Все население"], errors="ignore")
    # словарь переименования
    rename_map = {
        "Дети в возрасте до 16 лет": "proc_child",
        "Население старше трудоспособного возраста": "proc_pension",
        "Население трудоспособного возраста": "proc_work",
    }

    # применяем
    df = df.rename(columns=rename_map)
    return df


ps_2017 = rename_cols_ps(ps_2017)
ps_2018 = rename_cols_ps(ps_2018)
ps_2019 = rename_cols_ps(ps_2019)
ps_2020 = rename_cols_ps(ps_2020)

In [20]:
# Соединим таблицы в одну, функция добавляет номер года перед названием признака
def add_year_prefix(df: pd.DataFrame, year: int, exclude: list = None) -> pd.DataFrame:
    if exclude is None:
        exclude = []
    
    rename_map = {col: f"{year}_{col}" for col in df.columns if col not in exclude}
    return df.rename(columns=rename_map)

# Применим функцию
ps_2017 = add_year_prefix(ps_2017, 2017, exclude=["region"])
ps_2018 = add_year_prefix(ps_2018, 2018, exclude=["region"])
ps_2019 = add_year_prefix(ps_2019, 2019, exclude=["region"])
ps_2020 = add_year_prefix(ps_2020, 2020, exclude=["region"])

# Соединяем по горизонтали (по ключу region), на данном этапе используем тип outer
ps = ps_2017.merge(ps_2018, on="region", how="outer").merge(ps_2019, on="region", how="outer").merge(ps_2020, on="region", how="outer")
ps.head()

,region,2017_proc_child,2017_proc_pension,2017_proc_work,2018_proc_child,2018_proc_pension,2018_proc_work,2019_proc_child,2019_proc_pension,2019_proc_work,2020_proc_child,2020_proc_pension,2020_proc_work
0,Архангельская область (без АО),37.4,7.1,55.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Архангельская область (кроме Ненец...,NaN,NaN,NaN,38.8,6.6,54.6,31.3,8.9,59.8,28.5,7.2,64.3
2,Ненецкий автономный округ (Арханге...,50.5,1.0,48.4,50.5,5.5,44.0,45.6,2.0,52.4,NaN,NaN,NaN
3,Тюменская область (без АО),52.5,2.2,45.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Тюменская область (кроме Ханты-Ман...,NaN,NaN,NaN,52.2,0.6,47.2,43.7,7.4,48.9,37.9,7.5,54.7


### 1.1.7. Также у нас есть 2 датасета с одной дополнительной колонной, которая описывает тип товара, произведённого в данном регионе, а статистические данные по годам описывают объём их производства на импорт

> Преобразуем их по аналогии с датасетом ***mad***

In [21]:
# отбираем только числовые колонки (годы)
year_cols = [c for c in rp_05_16.columns if isinstance(c, int) or str(c).isdigit()]

# группировка по регионам и суммирование по годам
rp = rp_05_16.groupby("region")[year_cols].sum().reset_index()

# группировка по регионам и суммирование по годам

# отбираем только числовые колонки (годы)
year_cols = [c for c in rp_17_20.columns if isinstance(c, int) or str(c).isdigit()]
rp_17_20 = rp_17_20.groupby("region")[year_cols].sum().reset_index()

rp = rp.merge(rp_17_20, on = 'region', how = 'left')

## 1.2. Очистка данных 

На базе табицы *'Среднедушевые денежные доходы (в месяц), руб.'* - **pcci** выполним очитску данных:

- Удалим лишние строки (суммы по округам и областям), оставим Республики, области, и 4 основных округа

- Удалим лишние округа (например 'Приволжский федеральный округ' на мне нужен, так как все данные по входящим в него областям унас записаны в отдельных строках)

- Выполним нормализацию строк (приведение к нижнему индексу, удаление пробелов, переносов и тп)

- создадим список основных регионов [regions], по которому будем фильровать остальные датасеты

In [22]:
# Подгружаем список субъектов РФ, скаченный с официального сайта

regions = pd.read_excel('data/27572_справочник субъекты российской федерации.xlsx', skiprows=2)
regions = regions.rename(columns={'Наименование субъекта Российской Федерации ':'region'})

# Удаляем спорные териитории, т.к. их нет ни в одной статистике
regions = regions.drop(regions.index[85:89])
regions = regions.drop(['№ п/п', 'Код субъекта РФ'], axis = 1)

In [23]:
# Функция номрализации и унификации названий регионов в кадом датасете

_ZW_RE = re.compile(r"[\u200B-\u200D\uFEFF]")        # zero-width - невидимые символы нулевой ширины
_HYPHENS = "\u2010\u2011\u2012\u2013\u2014\u2212"    # строка со списком альтернативных символов дефисов/минусов, которые могут встретиться в тексте:

def _normalize_region(s: str) -> str:
    
    if s is None:
        return ""
    s = str(s)

    # Unicode нормализация и зачистка невидимых
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00A0", " ")         # NBSP -> пробел
    s = _ZW_RE.sub("", s)                # убрать zero-width
    s = s.translate({ord(ch): "-" for ch in _HYPHENS})

    # базовая подготовка
    s = s.strip().replace("\n", " ")
    s = re.sub(r'(?<=[а-яa-z])([А-ЯA-Z])', r' \1', s)  # пробел перед заглавной буквой
    s = s.replace("ё", "е").replace("Ё", "Е")
    s = s.lower().strip()

    # латиница -> похожие кириллические
    s = (s.replace("a","а").replace("t","т").replace("h","н").replace("b","в")
           .replace("k","к").replace("p","р").replace("x","х").replace("c","с")
           .replace("o","о").replace("e","е").replace("m","м"))

    # сохранить "_" при очистке
    s = re.sub(r"[^a-zа-я0-9\s\-_]", " ", s)

    # нормализация разделителей -> "_"
    s = re.sub(r"[\s\-]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")

    # расширение типовых сокращений
    s = re.sub(r'(?:(?<=^)|(?<=[_]))ао(?=(?:[_]|$))', 'автономный_округ', s)
    s = re.sub(r'(?:(?<=^)|(?<=[_]))авт_?округ(?=(?:[_]|$))', 'автономный_округ', s)
    
    # точечные правки
    s = s.replace("ямало_ненецкий_авт_округ", "ямало_ненецкий_автономный_округ")
    s = s.replace("ханты_мансийский_авт_округ", "ханты_мансийский_автономный_округ_югра")
    s = s.replace("еврейская_авт_область", "еврейская_автономная_область")
    s = s.replace("ненецкий_авт_округ", "ненецкий_автономный_округ")
    s = s.replace("чукотский_авт_округ", "чукотский_автономный_округ")
    s = s.replace("республика_татарстан_татарстан", "республика_татарстан")
    s = s.replace("чувашская_республика_чувашия", "чувашская_республика")

    # города федерального значения 
    if "москва" in s:           
        return "москва"
    if "санкт" in s:            
        return "санкт_петербург"
    if "севастоп" in s:         
        return "севастополь"

    return s

In [24]:
regions["region"] = regions["region"].apply(_normalize_region)
regions = list(regions['region'].value_counts().index)

In [25]:
regions[:5]
print(len(regions))

85


> Итого имеем 85 эталонных регионов

In [26]:
# Словарь из датасетов, которые необходимо очистить

datasets = {
    'pcci' : pcci,
    'ri' : ri,
    'fwp' : fwp,
    'rpay' : rpay,
    'cmr' : cmr,
    'cmu' : cmu,
    'dta' : dta_out,
    'alco' : alco,
    'drugs' : drugs,
    'grp' : grp,
    'hc' : hc,
    'hi' : hi,
    'mad' : mad_wide,
    'nm' : nm,
    'pop' : pop,
    'ppr' : ppr_wide,
    'ps' : ps,
    'rp' : rp,
    'rtpc' : rtpc,
    'wes' : wes
}

In [27]:
# Функция нормализации словаря из датасетов

# Применяет _normalize_region к колонке column во всех датасетах.

def normalize_datasets(datasets: dict, column: str) -> dict:
    normed = {}
    for name, df in datasets.items():
        if column not in df.columns or df.empty:
            continue
        new_df = df.copy()
        new_df[column] = new_df[column].astype(str).apply(_normalize_region)
        normed[f"norm_{name}"] = new_df.reset_index(drop=True)
    return normed

In [28]:
# Функция отрезки 'хвостов'

def strip_suffix(val: str, canon_list: list) -> str:
    # Если строка начинается с канона + '_...' — вернём чистый канон
    for ref in canon_list:
        if val == ref:
            return val
        if val.startswith(ref + "_"):
            return ref
    return val

# Возвращает словарь с обрезанными «хвостами» после канона.
def apply_strip_suffix(datasets: dict, column: str, canon_list: list) -> dict:
    out = {}
    for name, df in datasets.items():
        if column not in df.columns or df.empty:
            continue
        new_df = df.copy()
        new_df[column] = new_df[column].astype(str).apply(lambda v: strip_suffix(v, canon_list))
        out[name] = new_df.reset_index(drop=True)
    return out

In [29]:
# АЛИАСЫ (СИНОНИМЫ -> КАНОН)

ALIASES = {
    "ханты_мансийский_автономный_округ": "ханты_мансийский_автономный_округ_югра",
    "республика_адыгея_адыгея": "республика_адыгея",
}

def apply_aliases(datasets: dict, column: str, aliases: dict) -> dict:
    # Возвращает словарь: заменяет значения по алиасам (после нормализации!).
    out = {}
    for name, df in datasets.items():
        if column not in df.columns or df.empty:
            continue
        new_df = df.copy()
        new_df[column] = new_df[column].astype(str).map(lambda v: aliases.get(v, v))
        out[name] = new_df.reset_index(drop=True)
    return out

In [30]:
# Проверка несовпадений

# какие каноны не требовать в отдельных датасетах
IGNORE_BY_DATASET = {
    # датасеты alco_05_16, drugs_05_16 — Крым/Севастополь отсутствуют
    "norm_alco_05_16": {"республика_крым", "севастополь"},
    "norm_drugs_05_16": {"республика_крым", "севастополь"},
}

def find_missing_refs_full(datasets: dict, column: str, ref_regions: list, ignore_by_dataset: dict | None = None) -> dict:
    # Полное совпадение: возвращает {dataset: [каноны, которых нет]}. Предполагается, что и датасеты, и ref_regions уже нормализованы/приведены
    ref_set = set(ref_regions)
    result = {}
    for name, df in datasets.items():
        if column not in df.columns or df.empty:
            continue
        ignore = set() if not ignore_by_dataset else ignore_by_dataset.get(name, set())
        must_have = ref_set - ignore
        have = set(df[column].astype(str))
        missing = sorted(must_have - have)
        if missing:
            result[name] = missing
    return result

In [31]:
# 2) нормализуем датасеты
norm_datasets = normalize_datasets(datasets, "region")

# 3) обрезаем «хвосты» после канона
norm_datasets = apply_strip_suffix(norm_datasets, "region", regions)

# 4) применяем алиасы
norm_datasets = apply_aliases(norm_datasets, "region", ALIASES)

# 5) финальная проверка (с игнором по набором)
missing = find_missing_refs_full(norm_datasets, "region", regions, IGNORE_BY_DATASET)

# 6) вывод результата
for name, miss in missing.items():
    print(f"\n{name}:")
    for m in miss:
        print(" ", m)


norm_alco:
  республика_крым
  севастополь

norm_drugs:
  республика_крым
  севастополь


> Итого в признаке **missing** несовпадения по Республике Крым и г. Севастополь, так как данные регионы остутствуют в статситике

In [32]:
# Функция очитски датасетов

def build_clean_datasets(datasets: dict, regions: list, column: str = "region"):

    ref_set = set(regions)
    clean_datasets = {}

    for name, df in datasets.items():
        
        new_df = df.copy()
        
        # фильтрация по эталону
        new_df = new_df[new_df[column].isin(ref_set)].reset_index(drop=True)

        clean_datasets[f"clean_{name}"] = new_df

    return clean_datasets, regions

In [33]:
# Вызов функции
clean_datasets, regions = build_clean_datasets(norm_datasets, regions)

# распаковываем словарь в отдельные переменные
for var_name, df in clean_datasets.items():
    globals()[var_name] = df

In [34]:
# Финальная проверка датасетов
for name, df in clean_datasets.items():
    if "region" not in df.columns or df.empty:
        print(f"{name}: пустой или нет колонки 'region'")
        continue

    n_rows = len(df)
    n_unique = df["region"].nunique()
    print(f"{name}: {n_rows} строк, {n_unique} уникальных регионов")

clean_norm_pcci: 87 строк, 85 уникальных регионов
clean_norm_ri: 87 строк, 85 уникальных регионов
clean_norm_fwp: 87 строк, 85 уникальных регионов
clean_norm_rpay: 87 строк, 85 уникальных регионов
clean_norm_cmr: 87 строк, 85 уникальных регионов
clean_norm_cmu: 87 строк, 85 уникальных регионов
clean_norm_dta: 85 строк, 85 уникальных регионов
clean_norm_alco: 83 строк, 83 уникальных регионов
clean_norm_drugs: 83 строк, 83 уникальных регионов
clean_norm_grp: 87 строк, 85 уникальных регионов
clean_norm_hc: 85 строк, 85 уникальных регионов
clean_norm_hi: 85 строк, 85 уникальных регионов
clean_norm_mad: 87 строк, 85 уникальных регионов
clean_norm_nm: 88 строк, 85 уникальных регионов
clean_norm_pop: 87 строк, 85 уникальных регионов
clean_norm_ppr: 87 строк, 85 уникальных регионов
clean_norm_ps: 89 строк, 85 уникальных регионов
clean_norm_rp: 90 строк, 85 уникальных регионов
clean_norm_rtpc: 89 строк, 85 уникальных регионов
clean_norm_wes: 85 строк, 85 уникальных регионов


> В некоторых датастах количество регионов превышает 85, значит у нас есть дубли, необходимо их найти и удалить

In [35]:
# Функция фильтрации дубликатов, оставляем самые заполненные

def dedupe_keep_filled(df, col="region"):
    cols = [c for c in df.columns if c != col]
    filled = df[cols].notna().sum(axis=1)  # сколько непустых в строке
    idx = filled.groupby(df[col]).idxmax() # индексы самых «полных» строк
    return df.loc[idx].reset_index(drop=True)

# прогоняем по словарю
for name, df in list(clean_datasets.items()):
    if "region" in df.columns and 85 <= len(df) <= 90:
        before = len(df)
        clean_datasets[name] = dedupe_keep_filled(df, "region")
        after = len(clean_datasets[name])
        if name in globals():
            globals()[name] = clean_datasets[name]
        print(f"{name}: {before} → {after} строк")

clean_norm_pcci: 87 → 85 строк
clean_norm_ri: 87 → 85 строк
clean_norm_fwp: 87 → 85 строк
clean_norm_rpay: 87 → 85 строк
clean_norm_cmr: 87 → 85 строк
clean_norm_cmu: 87 → 85 строк
clean_norm_dta: 85 → 85 строк
clean_norm_grp: 87 → 85 строк
clean_norm_hc: 85 → 85 строк
clean_norm_hi: 85 → 85 строк
clean_norm_mad: 87 → 85 строк
clean_norm_nm: 88 → 85 строк
clean_norm_pop: 87 → 85 строк
clean_norm_ppr: 87 → 85 строк
clean_norm_ps: 89 → 85 строк
clean_norm_rp: 90 → 85 строк
clean_norm_rtpc: 89 → 85 строк
clean_norm_wes: 85 → 85 строк


> Все дубликаты удалены, коилчество регионов совпадает с эталонным [regions], за исключением двух таблиц, где отсутствует Крым и Севастополь

На данный момент рабочие датасеты под именем ***[clean_norm_'df']***

## 1.3. Поиск пропусков

> Согласно тепловой карте большая часть всех пропусков в датасетах **[clean_norm_rp_05_16, clean_norm_rp_17_20]**

*Это связано с производимыми в регионах товарами, потому что в каждом регионе свой вид производства*

*В данных таблицах все пропуски заменим на 0* ---> по итогу поличим нулевое производство в тех регионах где оно отутствует

> Замена выполнена, повторно соединим таблицы, посмотрим на процент оставшихся пропусков

> Для более подробного анализа выполним функцию find_nantypes, которая ищет стркои с пропусками в датасетах и выводит их на экран

In [36]:
def find_nantypes(df):
    nantypes = {}
    for i in clean_datasets:
        if clean_datasets[i].isna().sum().sum() > 0:
            nantypes[i] = clean_datasets[i]
    print(f'Общее количество датасетов с пропусками {len(nantypes)},\nДатасаты:{list(nantypes.keys())}')
    for name, df in nantypes.items():
        # ищем строки с NaN
        rows_with_null = df[df.isnull().any(axis=1)]
        
        if not rows_with_null.empty:
            print(f"\n{name}: найдено {len(rows_with_null)} строк с пропусками")
            display(rows_with_null)
        else:
            print(f"\n{name}: пропусков нет")   

find_nantypes(clean_datasets)     


Общее количество датасетов с пропусками 13,
Датасаты:['clean_norm_ri', 'clean_norm_cmr', 'clean_norm_cmu', 'clean_norm_alco', 'clean_norm_drugs', 'clean_norm_grp', 'clean_norm_mad', 'clean_norm_nm', 'clean_norm_pop', 'clean_norm_ppr', 'clean_norm_ps', 'clean_norm_rp', 'clean_norm_rtpc']

clean_norm_ri: найдено 2 строк с пропусками


,region,2015,2016,2017,2018,2019,2020
52,республика_крым,NaN,102.1,106.4,105.9,99.0,99.5
67,севастополь,NaN,139.3,104.1,101.4,101.2,102.9



clean_norm_cmr: найдено 6 строк с пропусками


,region,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
29,москва,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,3.0,9.0,11.0,6.0,11.0,13.0,14.0,5.0,10.0,9.0
52,республика_крым,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,88.0,63.0,63.0,43.0,53.0,54.0,51.0
63,санкт_петербург,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,севастополь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,1.0,1.0,0.0,0.0,0.0
78,ханты_мансийский_автономный_округ_югра,44.0,44.0,40.0,46.0,25.0,47.0,31.0,34.0,33.0,...,15.0,9.0,10.0,10.0,8.0,15.0,5.0,4.0,5.0,NaN
80,чеченская_республика,662.0,660.0,510.0,0.0,0.0,0.0,0.0,NaN,NaN,...,439.0,269.0,319.0,244.0,190.0,171.0,137.0,99.0,72.0,81.0



clean_norm_cmu: найдено 4 строк с пропусками


...

,region,2017_proc_child,2017_proc_pension,2017_proc_work,2018_proc_child,2018_proc_pension,2018_proc_work,2019_proc_child,2019_proc_pension,2019_proc_work,2020_proc_child,2020_proc_pension,2020_proc_work
4,белгородская_область,43.4,11.8,44.8,44.8,12.9,42.3,0.0,0.0,0.0,NaN,NaN,NaN
6,владимирская_область,34.8,8.6,56.6,48.9,8.3,42.8,34.8,4.8,60.4,NaN,NaN,NaN
12,ивановская_область,33.2,10.9,55.9,31.2,9.5,59.3,47.5,5.0,47.5,NaN,NaN,NaN
16,калужская_область,41.5,8.0,50.6,40.8,5.5,53.7,31.1,18.4,50.4,NaN,NaN,NaN
26,ленинградская_область,31.5,6.4,62.1,19.0,17.0,64.1,0.0,0.0,0.0,NaN,NaN,NaN
28,магаданская_область,32.2,14.8,53.0,30.7,11.5,57.8,45.8,0.0,54.2,NaN,NaN,NaN
32,ненецкий_автономный_округ,50.5,1.0,48.4,50.5,5.5,44.0,45.6,2.0,52.4,NaN,NaN,NaN
54,республика_мордовия,35.3,6.2,58.6,38.0,4.2,57.8,26.8,12.1,61.2,NaN,NaN,NaN
63,санкт_петербург,26.6,0.0,73.4,25.7,9.8,64.6,0.0,0.0,0.0,NaN,NaN,NaN
65,сахалинская_область,36.4,6.4,57.1,32.9,4.3,62.8,44.9,12.1,43.0,NaN,NaN,NaN



clean_norm_rp: найдено 21 строк с пропусками


,region,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,алтайский_край,9.261917e+07,1.164848e+08,1.302860e+08,1.816552e+08,1.481336e+08,2.049478e+08,2.273109e+08,2.405824e+08,2.474189e+08,2.543375e+08,3.215263e+08,3.157819e+08,NaN,NaN,NaN,NaN
1,амурская_область,3.181472e+07,3.586129e+07,3.875065e+07,4.679807e+07,6.086696e+07,7.210232e+07,9.669124e+07,1.024127e+08,9.924642e+07,1.093931e+08,1.386028e+08,1.259368e+08,NaN,NaN,NaN,NaN
10,еврейская_автономная_область,3.363165e+06,3.657249e+06,5.184157e+06,6.109987e+06,5.341850e+06,6.149376e+06,5.975404e+06,7.060658e+06,7.483082e+06,9.452933e+06,1.047541e+07,1.075672e+07,NaN,NaN,NaN,NaN
11,забайкальский_край,2.502080e+07,2.953846e+07,3.187896e+07,3.920246e+07,4.226029e+07,6.509903e+07,7.305327e+07,8.657463e+07,9.383908e+07,9.580649e+07,1.122074e+08,1.322210e+08,NaN,NaN,NaN,NaN
13,иркутская_область,2.043903e+08,2.652170e+08,2.782440e+08,3.099619e+08,3.143458e+08,4.185728e+08,5.104758e+08,5.825339e+08,6.369368e+08,7.094559e+08,8.984690e+08,9.513450e+08,NaN,NaN,NaN,NaN
17,камчатский_край,2.084657e+07,2.356995e+07,2.817257e+07,2.845628e+07,3.117140e+07,5.862333e+07,6.155174e+07,6.563864e+07,6.457302e+07,6.731731e+07,9.985023e+07,1.213378e+08,NaN,NaN,NaN,NaN
19,кемеровская_область,3.916558e+08,4.170390e+08,5.123187e+08,6.902343e+08,5.474250e+08,7.678539e+08,9.793549e+08,9.443497e+08,9.100352e+08,9.972943e+08,1.123950e+09,1.229405e+09,NaN,NaN,NaN,NaN
23,красноярский_край,3.728276e+08,5.152586e+08,6.122297e+08,5.685586e+08,5.823133e+08,8.979443e+08,9.901814e+08,9.838628e+08,1.060489e+09,1.230517e+09,1.447068e+09,1.508429e+09,NaN,NaN,NaN,NaN
28,магаданская_область,2.161007e+07,2.235134e+07,2.201160e+07,2.503819e+07,3.105893e+07,4.064902e+07,5.541893e+07,7.355060e+07,6.894998e+07,7.748592e+07,9.898321e+07,1.251508e+08,NaN,NaN,NaN,NaN
35,новосибирская_область,1.437600e+08,1.813816e+08,2.031832e+08,2.563956e+08,2.474128e+08,3.051104e+08,3.310820e+08,3.676408e+08,3.923451e+08,4.148353e+08,5.197738e+08,5.042132e+08,NaN,NaN,NaN,NaN



clean_norm_rtpc: найдено 2 строк с пропусками


,region,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
52,республика_крым,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,79010,112916,115133,120177,133940,143660,148071,176044
67,севастополь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90459,94360,145575,146264,149255,154734,141383,187776



>- для регионов выполним замену пропусков(для крайних ---> предыдущее значения, для центральных ---> интерполяция)


In [37]:
def impute_timeseries(
    datasets: dict,
    id_col: str = "region",
    ignore: set = {},
) -> dict:
    imputed = {}

    for name, df in datasets.items():
        if name in ignore:
            # просто оставляем как есть
            cleaned = df.copy()
        else:
            # выбираем числовые колонки (годы)
            ycols = [
                c for c in df.columns
                if c != id_col and (str(c).isdigit() or pd.api.types.is_numeric_dtype(df[c]))
            ]

            cleaned = df.copy()
            cleaned[ycols] = cleaned[ycols].apply(pd.to_numeric, errors="coerce")

            # интерполяция + заполнение краёв
            cleaned[ycols] = cleaned[ycols].T.interpolate("linear", limit_area="inside").T
            cleaned[ycols] = cleaned[ycols].ffill(axis=1).bfill(axis=1)

        imputed[name] = cleaned
        globals()[name] = cleaned

    # обновляем исходный словарь
    datasets.update(imputed)
    return imputed

In [38]:
filtred_datasets = impute_timeseries(clean_datasets)
find_nantypes(filtred_datasets)

Общее количество датасетов с пропусками 0,
Датасаты:[]


## 1.4. Преобразование признаков

> Посчитаем среднее по статистическим показателям

> В датасетах ***[_ri, _rpay]*** считаем накопленный индекс

In [39]:
new_datasets = filtred_datasets.copy()

In [40]:
list_mean= ['pcci','fwp', 'cmr', 'cmu', 'grp', 'mad', 'nm', 'pop', 'ppr', 'ppr', 'rp', 'rtpc', 'wes', 'alco', 'drugs']

def add_last5_mean(new_datasets: dict, list_mean: list) -> dict:

    for name, df in new_datasets.items():
        for suffix in list_mean:
            if name.endswith(suffix):
                new_df = df.copy()

                # если есть region → делаем индексом
                if "region" in new_df.columns:
                    new_df = new_df.set_index("region")

                # последние 5 колонок
                last5 = new_df.columns[-5:]
                suffix_clean = suffix.replace('clean_norm_', '')
                # добавляем среднее
                new_df[f"{suffix_clean}_mean"] = new_df[last5].mean(axis=1)

                # обновляем словарь
                new_datasets[name] = new_df
                break

    return new_datasets

add_last5_mean(new_datasets, list_mean)
print(new_datasets.keys())

dict_keys(['clean_norm_pcci', 'clean_norm_ri', 'clean_norm_fwp', 'clean_norm_rpay', 'clean_norm_cmr', 'clean_norm_cmu', 'clean_norm_dta', 'clean_norm_alco', 'clean_norm_drugs', 'clean_norm_grp', 'clean_norm_hc', 'clean_norm_hi', 'clean_norm_mad', 'clean_norm_nm', 'clean_norm_pop', 'clean_norm_ppr', 'clean_norm_ps', 'clean_norm_rp', 'clean_norm_rtpc', 'clean_norm_wes'])


In [41]:
def add_mean_features(df, prefix, groups, years, id_col="region"):
    df = df.copy().set_index(id_col)

    for g in groups:
        new_col = f"{prefix}_mean{g}"
        # удаляем старый столбец, если был
        if new_col in df.columns:
            df = df.drop(columns=[new_col])

        cols = [f"{y}{g}" for y in years if f"{y}{g}" in df.columns]
        if cols:
            df[new_col] = (
                pd.to_numeric(df[cols].stack(), errors="coerce")
                  .groupby(level=0).mean().round()
            )

    return df

# применение для dta
new_datasets["clean_norm_dta"] = add_mean_features(
    new_datasets["clean_norm_dta"],
    prefix="dta",
    groups=['_до_60', '_после_60'],
    years=range(2017, 2023)
)

# применение для ps
new_datasets["clean_norm_ps"] = add_mean_features(
    new_datasets["clean_norm_ps"],
    prefix="ps",
    groups=['_proc_child', '_proc_pension', '_proc_work'],
    years=range(2017, 2021)
)

In [42]:
# Расчёт накопительного индекса 2015-2020 гг

cols = new_datasets['clean_norm_ri'].columns[1:]
new_datasets['clean_norm_ri'][cols]
ri_cum = (new_datasets['clean_norm_ri'][cols] / 100).cumprod(axis=1) * 100
new_datasets['clean_norm_ri']['ri_cum'] = round(ri_cum[2020],1)

new_datasets['clean_norm_ri'] = new_datasets['clean_norm_ri'].set_index('region')

cols = new_datasets['clean_norm_rpay'].columns[1:]
new_datasets['clean_norm_rpay'][cols]
rpay_cum = (new_datasets['clean_norm_rpay'][cols] / 100).cumprod(axis=1) * 100
new_datasets['clean_norm_rpay']['rpay_cum'] = round(rpay_cum[2020],1)

new_datasets['clean_norm_rpay'] = new_datasets['clean_norm_rpay'].set_index('region')

In [43]:
# Удаляем все колонны кроме mean и cum

for name, df in new_datasets.items():
    if name == 'clean_norm_hc' or name == 'clean_norm_hi':
        new_datasets[name] = new_datasets[name].set_index('region')
    else:
        new_datasets[name] = new_datasets[name].filter(regex =  r"(mean|cum)")

## 1.5. Соединение таблиц

In [44]:
def merge_clean_datasets(datasets: dict, prefix_to_strip: str = "clean_norm_", id_col: str = "region") -> pd.DataFrame:
    frames = []
    for name, df in datasets.items():
        if df is None or df.empty:
            continue
        prefix = name.removeprefix(prefix_to_strip)
        temp = df.copy()

        # region → index
        if id_col in temp.columns:
            temp = temp.set_index(id_col)
        temp.index.name = id_col

        # дубликаты регионов → берём первую строку
        temp = temp[~temp.index.duplicated(keep="first")]

        # префикс к колонкам
        temp.columns = [f"{c}" for c in temp.columns if c != id_col]
        frames.append(temp)

    return pd.concat(frames, axis=1, join="outer").sort_index() if frames else pd.DataFrame()

In [45]:
full_df = merge_clean_datasets(new_datasets)

In [46]:
# Повторная проверка пропусков
full_df.isna().sum()

pcci_mean               0
ri_cum                  0
fwp_mean                0
rpay_cum                0
cmr_mean                0
cmu_mean                0
dta_mean_до_60          0
dta_mean_после_60       0
alco_mean               2
drugs_mean              2
grp_mean                0
hc_proc_НИС             0
hc_ЖП                   0
hc_proc_ИС              0
hi_proc_НУЖУ            0
hi_proc_УЖУ             0
mad_mean                0
nm_mean                 0
pop_mean                0
ppr_mean                0
ps_mean_proc_child      0
ps_mean_proc_pension    0
ps_mean_proc_work       0
rp_mean                 0
rtpc_mean               0
wes_mean                0
dtype: int64

> После соединения таблиц остались пропуски в *[alco_mean, drugs_mean]*

In [47]:
# Пропуски в г. Севастополь и Республика_крым, т.к. в drugs и alco нет данных по ним
# Заменим средним занчением по колонне

full_df[["alco_mean", "drugs_mean"]] = full_df[["alco_mean", "drugs_mean"]].apply(lambda x: x.fillna(x.mean()))

## 1.6. Корректировка колонок population (pop_)

> В колонке [pop_mean] присутствуют аномальные занчения, например в строках (***иркутская_область, кировская_область, красноярский_край, 	новосибирская_область, приморский_край)*** аномально низкие значения, а в строке ***белгородская область*** аномально высокие значения.

Для общей проверки посчитаем сумму населения по строкам, по данным росстата **оринетировочная численность населения за 2022 год - 147 млн. чел.**

In [48]:
full_df['pop_mean'].sum()

np.float64(259709425.6)

> Общая численность населения аномально высокая на **100 млн чел. выше статистики**

Для анализа подгрузим таблицу численности населения по субъектам РФ за 2023 год с сайта Росстата:
http://ssl.rosstat.gov.ru/storage/mediabank/BUL_MO_2023.xlsx

Выполним преобразование датасета по аналогии с [1]


In [49]:
pop_df = pd.read_excel('data/BUL_MO_2023.xlsx', sheet_name=3, skiprows=4)
pop_df = pop_df.rename(columns={'Unnamed: 0':'region'})
pop_df = pop_df.rename(columns={'Все население':'count'})
pop_df = pop_df[['region', 'count']]
pop_dict = {'pop_df' : pop_df}

# 2) нормализуем датасеты
pop_dict_2023 = normalize_datasets(pop_dict, "region")

# 3) обрезаем «хвосты» после канона
pop_dict_2023 = apply_strip_suffix(pop_dict_2023, "region", regions)

# 4) применяем алиасы
pop_dict_2023 = apply_aliases(pop_dict_2023, "region", ALIASES)

# 5) финальная проверка (с игнором по набором)
missing = find_missing_refs_full(pop_dict_2023, "region", regions, IGNORE_BY_DATASET)

# 6) вывод результата
for name, miss in missing.items():
    print(f"\n{name}:")
    for m in miss:
        print(" ", m)

# Все совпадения найдены

clean_pop_2023, regions = build_clean_datasets(pop_dict_2023, regions)

pop_2023 = clean_pop_2023['clean_norm_pop_df']

def pop23_keep_filled(df, col="region"):
    df = df.rename(columns={'count' : 'pop_2023'})
    filled = df.notna().sum(axis=1)  # сколько непустых в строке
    idx = filled.groupby(df[col]).idxmax() # индексы самых «полных» строк
    return df.loc[idx].reset_index(drop=True)

before = len(pop_2023)
pop_2023 = pop23_keep_filled(pop_2023, "region")
after = len(pop_2023)
print(f"Было {before} → стало {after} строк")

Было 87 → стало 85 строк


In [50]:
# Добавим данные в полный датасет

full_df = full_df.merge(pop_2023, on = 'region', how = 'left')

# Суммарное количетсво населения по субъектам РФ
full_df['pop_2023'].sum()

np.int64(148731547)

> **Скачанные данные актуальны - общая сумма населения соответствует статиситческим данным**

> Удалим колонну численности населения [pop_mean], вместо нее будем использовать pop_2023

In [51]:
full_df = full_df.drop('pop_mean', axis = 1)

In [52]:
full_df.shape

(85, 27)

In [53]:
full_df.to_csv('data/rosstat_data.csv')

### Очистка и замена пропусков выполнена успешно ---> имеем полный датасет из 85 строк и 30 колонн

# Выводы по разделу 1:

- Выполнена очистка данных:
    - колонки, содержащие наименования субъектов РФ приведены к единому формату [region];
    - исключены лишние регионы, автономные округа, области РФ. 
    - оставшиеся субъекты РФ соответствуют [Справочнику субъектов Российской федерации](https://docs.yandex.ru/docs/view?tm=1756977946&tld=ru&lang=ru&name=27571_%D1%81%D0%BF%D1%80%D0%B0%D0%B2%D0%BE%D1%87%D0%BD%D0%B8%D0%BA%20%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B%20%D1%80%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9%20%D1%84%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8.pdf&text=%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5%20%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B%20%D0%A0%D0%A4%20%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA&url=https%3A%2F%2Fszfo.fsrar.gov.ru%2Ffiles%2F27571_%25D1%2581%25D0%25BF%25D1%2580%25D0%25B0%25D0%25B2%25D0%25BE%25D1%2587%25D0%25BD%25D0%25B8%25D0%25BA%2520%25D1%2581%25D1%2583%25D0%25B1%25D1%258A%25D0%25B5%25D0%25BA%25D1%2582%25D1%258B%2520%25D1%2580%25D0%25BE%25D1%2581%25D1%2581%25D0%25B8%25D0%25B9%25D1%2581%25D0%25BA%25D0%25BE%25D0%25B9%2520%25D1%2584%25D0%25B5%25D0%25B4%25D0%25B5%25D1%2580%25D0%25B0%25D1%2586%25D0%25B8%25D0%25B8.pdf&lr=213&mime=pdf&l10n=ru&sign=61d4c7951abeb54bac668650c14e280f&keyno=0&serpParams=tm%3D1756977946%26tld%3Dru%26lang%3Dru%26name%3D27571_%25D1%2581%25D0%25BF%25D1%2580%25D0%25B0%25D0%25B2%25D0%25BE%25D1%2587%25D0%25BD%25D0%25B8%25D0%25BA%2520%25D1%2581%25D1%2583%25D0%25B1%25D1%258A%25D0%25B5%25D0%25BA%25D1%2582%25D1%258B%2520%25D1%2580%25D0%25BE%25D1%2581%25D1%2581%25D0%25B8%25D0%25B9%25D1%2581%25D0%25BA%25D0%25BE%25D0%25B9%2520%25D1%2584%25D0%25B5%25D0%25B4%25D0%25B5%25D1%2580%25D0%25B0%25D1%2586%25D0%25B8%25D0%25B8.pdf%26text%3D%25D0%25BE%25D1%2581%25D0%25BD%25D0%25BE%25D0%25B2%25D0%25BD%25D1%258B%25D0%25B5%2B%25D1%2581%25D1%2583%25D0%25B1%25D1%258A%25D0%25B5%25D0%25BA%25D1%2582%25D1%258B%2B%25D0%25A0%25D0%25A4%2B%25D1%2581%25D0%25BF%25D0%25B8%25D1%2581%25D0%25BE%25D0%25BA%26url%3Dhttps%253A%2F%2Fszfo.fsrar.gov.ru%2Ffiles%2F27571_%2525D1%252581%2525D0%2525BF%2525D1%252580%2525D0%2525B0%2525D0%2525B2%2525D0%2525BE%2525D1%252587%2525D0%2525BD%2525D0%2525B8%2525D0%2525BA%252520%2525D1%252581%2525D1%252583%2525D0%2525B1%2525D1%25258A%2525D0%2525B5%2525D0%2525BA%2525D1%252582%2525D1%25258B%252520%2525D1%252580%2525D0%2525BE%2525D1%252581%2525D1%252581%2525D0%2525B8%2525D0%2525B9%2525D1%252581%2525D0%2525BA%2525D0%2525BE%2525D0%2525B9%252520%2525D1%252584%2525D0%2525B5%2525D0%2525B4%2525D0%2525B5%2525D1%252580%2525D0%2525B0%2525D1%252586%2525D0%2525B8%2525D0%2525B8.pdf%26lr%3D213%26mime%3Dpdf%26l10n%3Dru%26sign%3D61d4c7951abeb54bac668650c14e280f%26keyno%3D0) [источник : [szfo.fsrar.gov.ru](https://szfo.fsrar.gov.ru/)]

- Выполнена проверка и замена пропусков


- Удалены шумвые признаки, созданы признаки с осреднёнными сзначениями статистических данных по всем регионам

- Очищенный датасет сохранён в формат csv. ***Размер - [85, 27]***